In [4]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

def start_stable_driver():
    options = Options()
    options.page_load_strategy = 'eager'
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("detach", True)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    # Hız ve stabilite için resimleri yükleme
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Global Değişkenler
BASE_URL = "https://www.sikayetvar.com/arcelik/kurutma-makinesi"
dryer_links = []

In [5]:
target_pages = 137 
current_page = 1 
refresh_limit = 20 

print(f"🚀 Kurutma Makinesi linkleri toplanıyor...")

while current_page <= target_pages:
    driver = start_stable_driver()
    next_stop = min(current_page + refresh_limit - 1, target_pages)
    
    try:
        for p in range(current_page, next_stop + 1):
            success = False
            attempts = 0
            while not success and attempts < 2:
                try:
                    driver.get(f"{BASE_URL}?page={p}")
                    time.sleep(2.5)
                    
                    links = driver.find_elements(By.CSS_SELECTOR, "a.complaint-layer")
                    if len(links) > 0:
                        for l in links:
                            href = l.get_attribute("href")
                            title = l.get_attribute("title")
                            if href and "/arcelik/" in href:
                                if not any(d['url'] == href for d in dryer_links):
                                    dryer_links.append({"baslik": title, "url": href})
                        success = True
                    else:
                        time.sleep(5) # Captcha kontrolü
                        attempts += 1
                except:
                    attempts += 1
            
            if p % 10 == 0:
                print(f"✅ Sayfa {p} bitti. Havuz: {len(dryer_links)} link.")
                
            current_page = p + 1
    except:
        pass
    
    driver.quit()
    print("🧹 Bellek temizlendi, devam ediliyor...")

print(f"🏁 Link Toplama Bitti! Toplam {len(dryer_links)} link hazır.")

🚀 Kurutma Makinesi linkleri toplanıyor...
✅ Sayfa 10 bitti. Havuz: 231 link.
✅ Sayfa 20 bitti. Havuz: 460 link.
🧹 Bellek temizlendi, devam ediliyor...
✅ Sayfa 30 bitti. Havuz: 635 link.
✅ Sayfa 40 bitti. Havuz: 744 link.
🧹 Bellek temizlendi, devam ediliyor...
✅ Sayfa 50 bitti. Havuz: 832 link.
✅ Sayfa 60 bitti. Havuz: 900 link.
🧹 Bellek temizlendi, devam ediliyor...
✅ Sayfa 70 bitti. Havuz: 960 link.
✅ Sayfa 80 bitti. Havuz: 1019 link.
🧹 Bellek temizlendi, devam ediliyor...
✅ Sayfa 90 bitti. Havuz: 1055 link.
✅ Sayfa 100 bitti. Havuz: 1098 link.
🧹 Bellek temizlendi, devam ediliyor...
✅ Sayfa 110 bitti. Havuz: 1133 link.
✅ Sayfa 120 bitti. Havuz: 1167 link.
🧹 Bellek temizlendi, devam ediliyor...
✅ Sayfa 130 bitti. Havuz: 1201 link.
🧹 Bellek temizlendi, devam ediliyor...
🏁 Link Toplama Bitti! Toplam 1225 link hazır.


In [6]:
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=50, pool_maxsize=50)
session.mount('https://', adapter)

def fetch_dryer_ultra_compatible(item):
    url = item['url']
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    
    for attempt in range(5): # Deneme sayısını 5'e çıkardık
        try:
            time.sleep(random.uniform(0.5, 1.0))
            res = session.get(url, headers=headers, timeout=15)
            
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                content_parts = []
                
                # YÖNTEM 1: Modern Tasarım (data-ga etiketleri)
                start_node = soup.find("div", {"data-ga-element": "Complaint_Content_Start"})
                if start_node:
                    for sibling in start_node.find_next_siblings():
                        if sibling.name == "div" and sibling.get("data-ga-element") == "Complaint_Content_End": break
                        if sibling.name == "p": content_parts.append(sibling.get_text(strip=True))
                
                # YÖNTEM 2: Orta Nesil Tasarım (description divi)
                if not content_parts:
                    desc_div = soup.find("div", class_="complaint-detail-description")
                    if desc_div: content_parts = [desc_div.get_text(strip=True)]
                
                # YÖNTEM 3: Arşiv/Eski Tasarım (yalın p etiketleri)
                if not content_parts:
                    p_tags = soup.select(".description p") or soup.select(".complaint-detail-description p")
                    content_parts = [p.get_text(strip=True) for p in p_tags]

                full_text = " ".join(content_parts)
                if len(full_text) > 10:
                    return {"baslik": item["baslik"], "url": url, "aciklama": full_text}
            
            elif res.status_code == 429:
                time.sleep(15) # Hız sınırına takılırsa uzun mola
        except:
            continue
            
    return {"baslik": item["baslik"], "url": url, "aciklama": "HATA: Alınamadı."}

# --- ÇALIŞTIRICI ---
print(f"📥 {len(dryer_links)} adet kurutma makinesi şikayeti çekiliyor...")
final_dryer_results = []
start_time = time.time()

# Sunucuyu yormamak için max_workers=6 idealdir
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(fetch_dryer_ultra_compatible, link) for link in dryer_links]
    for i, future in enumerate(futures, 1):
        final_dryer_results.append(future.result())
        
        if i % 100 == 0:
            elapsed = time.time() - start_time
            rem = (elapsed / i) * (len(dryer_links) - i) / 60
            print(f"📊 İlerleme: {i}/{len(dryer_links)} | Kalan: ~{rem:.1f} dk")
            # Her 200'de bir yedek al
            pd.DataFrame(final_dryer_results).to_excel("arcelik_kurutma_GUNCEL_YEDEK.xlsx", index=False)

# Final Kayıt
pd.DataFrame(final_dryer_results).to_excel("arcelik_kurutma_makinesi_YENI_FINAL.xlsx", index=False)
print("✅ İŞLEM TAMAMLANDI!")

📥 1225 adet kurutma makinesi şikayeti çekiliyor...
📊 İlerleme: 100/1225 | Kalan: ~13.5 dk
📊 İlerleme: 200/1225 | Kalan: ~12.4 dk
📊 İlerleme: 300/1225 | Kalan: ~11.5 dk
📊 İlerleme: 400/1225 | Kalan: ~10.1 dk
📊 İlerleme: 500/1225 | Kalan: ~8.5 dk
📊 İlerleme: 600/1225 | Kalan: ~7.1 dk
📊 İlerleme: 700/1225 | Kalan: ~6.2 dk
📊 İlerleme: 800/1225 | Kalan: ~5.2 dk
📊 İlerleme: 900/1225 | Kalan: ~3.8 dk
📊 İlerleme: 1000/1225 | Kalan: ~2.6 dk
📊 İlerleme: 1100/1225 | Kalan: ~1.5 dk
📊 İlerleme: 1200/1225 | Kalan: ~0.3 dk
✅ İŞLEM TAMAMLANDI!
